![POLITICO](https://rawgithub.com/The-Politico/src/master/images/logo/badge.png)

# POLITICO demographic district similarity maps

POLITICO demographic district similarity maps create lists of the most similar districts to any one district based on census demographics.

The maps are created by calculating a weighted Euclidean distance between demographic characteristics in each district.